# Main

In [38]:
import dfBasics
import pandas as pd

version_sla = 'v00001'
version     = version_sla + '/v00000'

home_directory  =  '/home/jovyan/work/'
#share_directory =  '/home/jovyan/work/share/'
share_directory =  '/home/jovyan/share/'

In [5]:
sparkSession = dfBasics.getSparkSession()

In [6]:
#senders = list(sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/v00000/v00000/senders/senders.parquet').toPandas()['CSENDERENDPOINTID'])

In [7]:
senders_files = sparkSession.read.options(delimiter=',') \
                      .csv('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/senders/senders_files.txt').toPandas()
senders_files.columns = ['size','filename']    

# Examples

## some info

In [11]:
len(senders_files),max(senders_files['size'])

(697, '9772')

## get message tracking data 

In [8]:
def get_empty_message_tracking_dataframe():
    columns = ['CGLOBALMESSAGEID', 'CSTARTTIME', 'CENDTIME', 'CSTATUS', 'CSERVICE', \
       'CSENDERENDPOINTID', 'CSENDERPROTOCOL', 'CINBOUNDSIZE', \
       'CRECEIVERPROTOCOL', 'CRECEIVERENDPOINTID', 'CSLATAT', 'CMESSAGETAT2', \
       'CSLADELIVERYTIME', 'year', 'month', 'day', 'hour', 'minute']
    return pd.DataFrame(columns=columns)

def get_message_tracking_dataframe_of_senders_filename(filename, version=version):
    try:
        return sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + filename).toPandas()
    except Exception as exception: 
        return get_empty_message_tracking_dataframe()

In [9]:
pfall = get_message_tracking_dataframe_of_senders_filename(senders_files['filename'][0])    

In [10]:
pfall.head()

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERENDPOINTID,CSENDERPROTOCOL,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME,year,month,day,hour,minute
0,175cf1c1-4afa-11ed-be5c-0905ac1e100e,1665667351259,1665667357772,14,6,2,4,33672,0,4870,6466,6513,1665667357725,2022,10,13,15,22
1,ee445b1f-4afa-11ed-b5bc-15f0ac1e100d,1665667711809,1665667716813,14,6,2,4,33664,0,3932,4957,5004,1665667716766,2022,10,13,15,28
2,42d298e3-4ad9-11ed-b5bc-15f0ac1e100d,1665653250781,1665653255722,14,6,2,4,33820,0,2382,4909,4941,1665653255690,2022,10,13,11,27
3,6ca7539c-4ad8-11ed-be5c-0905ac1e100e,1665652891465,1665652897927,14,6,2,4,33659,0,3476,6415,6462,1665652897880,2022,10,13,11,21
4,6b5bcb58-4ad9-11ed-be5c-0905ac1e100e,1665653318789,1665653325777,14,6,2,4,33652,0,4102,6957,6988,1665653325746,2022,10,13,11,28


In [17]:
len(pd.unique(pfall['CGLOBALMESSAGEID'])),len(pfall), pd.unique(pfall['CSTATUS']), pd.unique(pfall['CSERVICE'])

(13, 13, array([14], dtype=int32), array([6], dtype=int32))

In [18]:
pfall[ pfall['CSERVICE'] == -1]

,CGLOBALMESSAGEID,CSTARTTIME,CENDTIME,CSTATUS,CSERVICE,CSENDERENDPOINTID,CSENDERPROTOCOL,CINBOUNDSIZE,CRECEIVERPROTOCOL,CRECEIVERENDPOINTID,CSLATAT,CMESSAGETAT2,CSLADELIVERYTIME,year,month,day,hour,minute


## check message tracking files

In [ ]:
def get_skip_senders(senders_files):
    skip_senders_files = []
    for file in senders_files:
        try:
            pfall = sparkSession.read.parquet('hdfs://172.30.17.145:8020/user/admin/sla/' + version + '/encoded/senders/' + file).toPandas()
            if -1 in pd.unique(pfall['CSENDERENDPOINTID']) :
                skip_senders_files.append(file)
        except Exception as exception:     
            skip_senders_files.append(file)

In [ ]:
skip_senders_files = get_skip_senders(senders_files['filename'])
tested_senders_files = [file for file in senders_files['filename'] if file not in skip_senders_files]
#len(skip_senders_files)

## inverse transform encoded values

In [1]:
# - run this paragraph only once !
# - if you get an 'allow_pickle' error you need Kernel/restart kernel

import encoder
import numpy as np

np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

# restore np.load for future normal usage
#np.load = np_load_old

In [36]:
def inverse_transform(value,column=None, npy= share_directory + 'sla/' + version_sla + '/npy'):
    _encoder = encoder.TolerantLabelEncoder(ignore_unknown=True)
    _encoder.classes_ = np.load(npy + '/' + column + '.npy')
    if type(value) == int:
        return str(_encoder.inverse_transform(value))  
    elif type(value) == list:
        return [str(_encoder.inverse_transform(v)) for v in value]
    else:
        return None

In [37]:
inverse_transform([6,5],column='CSERVICE')
inverse_transform(6, 'CSTATUS')

'ERROR'

## status : was the message processed successfully ?

In [ ]:
pfall = pfall.assign(outcome=(~( ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'InvoicePortal')) | ((pfall['CSTATUS'] == 'PENDING') & (pfall['CSERVICE'] == 'IDS')) | (pfall['CSTATUS'] == 'SUCCESS') | (pfall['CSTATUS'] == 'SUCCESS_DOWNLOADED') | (pfall['CSTATUS'] == 'SUCCESS_POLLQUEUE'))).astype(int))
    